<a href="https://colab.research.google.com/github/VasanthPrakasam/-NASA-Near-Earth-Object-NEO-Tracking-Insights-using-Public-API/blob/main/NASA(NEO)Data_Tracking_%26_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Pagination, Data Cleaning, & Data Processing

In [1]:
import requests

API_KEY = "RLlMuxVfiTbDAFG5TzMz4sJFr4t23afT76yE7jVa"

# Initialize an empty list to store the extracted asteroid data
asteroids_data = []

# Set your target number of asteroids

target = 10000  # You can change this to 10000 or any other number

# Initial URL for the first API call.
# Make sure the start_date and end_date are set reasonably for initial fetch.
# The API typically allows a maximum of 7 days between start_date and end_date.
# We'll use 7 days and let pagination handle the rest.
start_date = "2024-01-01"
end_date = "2024-01-07" # Max 7 days for initial feed request

In [2]:
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={API_KEY}" # f-string ~ PlaceHolder

s_no_counter = 1 # To keep track of the serial number for your desired output ~ Row Count

In [3]:
# Loop until we reach the target number of asteroids or run out of next pages
while len(asteroids_data) < target and url: # 'url' will become None when 'next' link is not available
    print(f"Fetching data from: {url}") # For debugging/progress
    try:
        response = requests.get(url) # success for 200
        response.raise_for_status() # Raise an exception for HTTP errors (400-client_Error or 500-Server_error)
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        break # Exit the loop on request error

    details = data.get('near_earth_objects')
    if not details:
        print("No 'near_earth_objects' found in the response for this page. Exiting.")
        break

    for date, asteroids_on_date in details.items():
        for ast in asteroids_on_date:
            # Safely get close_approach_data, as not all asteroids might have it
            close_approach_data = ast.get('close_approach_data')

            # Initialize values to None or default if close_approach_data is missing
            approach_date = None
            relative_velocity_kmph = None
            astronomical_distance = None
            miss_distance_km = None
            miss_distance_lunar = None
            orbiting_body = None

            if close_approach_data and len(close_approach_data) > 0:
                first_approach = close_approach_data[0]
                approach_date = first_approach.get('close_approach_date')
                relative_velocity_kmph = first_approach.get('relative_velocity', {}).get('kilometers_per_hour')
                astronomical_distance = first_approach.get('miss_distance', {}).get('astronomical')
                miss_distance_km = first_approach.get('miss_distance', {}).get('kilometers')
                miss_distance_lunar = first_approach.get('miss_distance', {}).get('lunar')
                orbiting_body = first_approach.get('orbiting_body')

            asteroid_info = {
                's.no': s_no_counter,
                'id': int(ast['id']),
                'neo_reference_id': ast['neo_reference_id'],
                'name': ast['name'],
                'absolute_magnitude_h': ast['absolute_magnitude_h'],
                'estimated_diameter_min_km': ast['estimated_diameter']['kilometers']['estimated_diameter_min'],
                'estimated_diameter_max_km': ast['estimated_diameter']['kilometers']['estimated_diameter_max'],
                'is_potentially_hazardous_asteroid': ast['is_potentially_hazardous_asteroid'],
                'close_approach_date': approach_date,
                'relative_velocity_kmph': float(relative_velocity_kmph) if relative_velocity_kmph else None,
                'astronomical': float(astronomical_distance) if astronomical_distance else None,
                'miss_distance_km': float(miss_distance_km) if miss_distance_km else None,
                'miss_distance_lunar': float(miss_distance_lunar) if miss_distance_lunar else None,
                'orbiting_body': orbiting_body
            }
            asteroids_data.append(asteroid_info)
            s_no_counter += 1

            if len(asteroids_data) >= target:
                break # Break from inner loop (asteroids on date)
        if len(asteroids_data) >= target:
            break # Break from outer loop (dates)

    # Get the URL for the next page, if available
    # If 'next' key doesn't exist, data.get('links', {}).get('next') will return None
    url = data.get('links', {}).get('next')

# Print the number of asteroids collected
print(f"\nCollected {len(asteroids_data)} asteroids (target was {target}).")

# Print the first 5 collected asteroids for verification
print("\nFirst 5 collected asteroids:")
for i, ast in enumerate(asteroids_data[:5]): #to get all data - [:]
    print(f"--- Asteroid {ast['s.no']} ---")
    for key, value in ast.items():
        print(f"{key}: {value}")
    print("-" * 20)

# The full list of collected asteroids is in 'asteroids_data'
# You can now process 'asteroids_data' further as needed.
# print(asteroids_data) # Uncomment to print the entire list

Fetching data from: https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key=RLlMuxVfiTbDAFG5TzMz4sJFr4t23afT76yE7jVa
Fetching data from: http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-07&end_date=2024-01-13&detailed=false&api_key=RLlMuxVfiTbDAFG5TzMz4sJFr4t23afT76yE7jVa
Fetching data from: http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-13&end_date=2024-01-19&detailed=false&api_key=RLlMuxVfiTbDAFG5TzMz4sJFr4t23afT76yE7jVa
Fetching data from: http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-19&end_date=2024-01-25&detailed=false&api_key=RLlMuxVfiTbDAFG5TzMz4sJFr4t23afT76yE7jVa
Fetching data from: http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-25&end_date=2024-01-31&detailed=false&api_key=RLlMuxVfiTbDAFG5TzMz4sJFr4t23afT76yE7jVa
Fetching data from: http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-31&end_date=2024-02-06&detailed=false&api_key=RLlMuxVfiTbDAFG5TzMz4sJFr4t23afT76yE7jVa
Fetching data from: http://api.nasa.

# Sql Insertion

In [4]:
!pip install pandas
!pip install mysql-connector-python
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [5]:
import sqlite3
connection=sqlite3.connect("Asteroid_Data.db")
cursor = connection.cursor()

In [6]:
#Outline -- Table Creation
# -- Table for Asteroids data
# -- This table would store general information about each asteroid.
# CREATE TABLE IF NOT EXISTS asteroids (
#     id int Not Unique --> Duplication ,
#     name TEXT NOT NULL,
#     absolute_magnitude_h REAL,
#     estimated_diameter_min REAL,
#     estimated_diameter_max REAL,
#     is_potentially_hazardous_asteroid BOOLEAN,
#     nasa_jpl_url TEXT,
#     sentry_object BOOLEAN
# );

# -------------------------------------------------
# -- Table for Close Approach data
# -- This table stores event-based data about each asteroid’s approach to Earth.
# CREATE TABLE IF NOT EXISTS close_approach (
#     -- Renamed from close_approaches to close_approach as per user's request
#     approach_id INTEGER PRIMARY KEY, -- A unique primary key for each close approach event
#     neo_reference_id INTEGER, -- References the 'id' in the 'asteroids' table
#     close_approach_date TEXT, -- Stored as TEXT for DATE, e.g., 'YYYY-MM-DD'
#     relative_velocity_kmph REAL,
#     astronomical_au REAL, -- Renamed from 'astronomical(AU)' to 'astronomical_au' for valid column name
#     miss_distance_km REAL,
#     miss_distance_lunar REAL,
#     orbiting_body TEXT, -- VARCHAR is TEXT in SQLite
#     FOREIGN KEY (neo_reference_id) REFERENCES asteroids(id)
# );


In [7]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS asteroids (
    id INTEGER,
    name TEXT NOT NULL,
    absolute_magnitude_h REAL,
    estimated_diameter_min_km REAL,
    estimated_diameter_max_km REAL,
    is_potentially_hazardous_asteroid BOOLEAN
)
''')
connection.commit()
# Don't forget to commit after creating tables

In [8]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS close_approach (
    neo_reference_id INTEGER,
    close_approach_date TEXT,
    relative_velocity_kmph REAL,
    astronomical REAL,
    miss_distance_km REAL,
    miss_distance_lunar REAL,
    orbiting_body TEXT,
    FOREIGN KEY (neo_reference_id) REFERENCES asteroids(id)
)
''')
connection.commit()

In [9]:
cursor.execute("SELECT * FROM asteroids")
data = cursor.fetchall()

#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [i[0] for i in cursor.description]


In [10]:

import pandas as pd

# using the result fetched in cursor, create a new dataframe

new_df = pd.DataFrame(data,columns=columns)

new_df # shows all the values inserted - 6columns

,id,name,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid


In [11]:
cursor.execute("SELECT * FROM close_approach")
data = cursor.fetchall()

#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [i[0] for i in cursor.description]


In [12]:

import pandas as pd

# using the result fetched in cursor, create a new dataframe

new_df = pd.DataFrame(data,columns=columns)

new_df # shows all the values inserted - 7 columns

,neo_reference_id,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body


In [13]:
asteroids_data[:2]

[{'s.no': 1,
  'id': 2415949,
  'neo_reference_id': '2415949',
  'name': '415949 (2001 XY10)',
  'absolute_magnitude_h': 19.37,
  'estimated_diameter_min_km': 0.3552670883,
  'estimated_diameter_max_km': 0.7944013596,
  'is_potentially_hazardous_asteroid': False,
  'close_approach_date': '2024-01-02',
  'relative_velocity_kmph': 57205.8951204341,
  'astronomical': 0.3372535274,
  'miss_distance_km': 50452409.349026635,
  'miss_distance_lunar': 131.1916221586,
  'orbiting_body': 'Earth'},
 {'s.no': 2,
  'id': 3160747,
  'neo_reference_id': '3160747',
  'name': '(2003 SR84)',
  'absolute_magnitude_h': 26.0,
  'estimated_diameter_min_km': 0.0167708462,
  'estimated_diameter_max_km': 0.0375007522,
  'is_potentially_hazardous_asteroid': False,
  'close_approach_date': '2024-01-02',
  'relative_velocity_kmph': 38589.054833182,
  'astronomical': 0.1323425924,
  'miss_distance_km': 19798169.933318187,
  'miss_distance_lunar': 51.4812684436,
  'orbiting_body': 'Earth'}]

In [14]:
# For i in asteroids values reference
	# id	name	absolute_magnitude_h	estimated_diameter_min_km	estimated_diameter_max_km	is_potentially_hazardous_asteroid --> 6 column


In [15]:
insert = "insert into asteroids values(?,?,?,?,?,?)"

for i in asteroids_data:
 values = (i['id'],i['name'],i['absolute_magnitude_h'],i['estimated_diameter_min_km'],i['estimated_diameter_max_km'],i['is_potentially_hazardous_asteroid'])
 cursor.execute(insert,values)
connection.commit()

In [16]:
# for i in close_approach values reference
# neo_reference_id	close_approach_date	relative_velocity_kmph	astronomical	miss_distance_km	miss_distance_lunar	orbiting_body --> 7 column

In [17]:
insert = "insert into close_approach values(?,?,?,?,?,?,?)"

for i in asteroids_data:
 values = (i['neo_reference_id'],i['close_approach_date'],i['relative_velocity_kmph'],i['astronomical'],i['miss_distance_km'],i['miss_distance_lunar'],i['orbiting_body'])
 cursor.execute(insert,values)
connection.commit()

In [18]:
cursor.execute("SELECT * FROM asteroids")
data = cursor.fetchall()
#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [i[0] for i in cursor.description]
import pandas as pd
# using the result fetched in cursor, create a new dataframe
new_df = pd.DataFrame(data,columns=columns)
new_df # shows all the values inserted - 7 columns

,id,name,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid
0,2415949,415949 (2001 XY10),19.370,0.355267,0.794401,0
1,3160747,(2003 SR84),26.000,0.016771,0.037501,0
2,3309828,(2005 YQ96),20.620,0.199781,0.446725,1
3,3457842,(2009 HC21),22.100,0.101054,0.225964,0
4,3553062,(2010 XA11),26.100,0.016016,0.035813,0
...,...,...,...,...,...,...
9995,54527449,(2025 HW2),22.335,0.090689,0.202787,0
9996,3152316,(2003 GJ21),22.900,0.069913,0.156329,0
9997,3441846,(2008 YN2),26.300,0.014607,0.032662,0
9998,3763283,(2016 VU1),24.600,0.031956,0.071456,0


In [19]:
cursor.execute("SELECT * FROM close_approach")
data = cursor.fetchall()
#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [i[0] for i in cursor.description]
import pandas as pd
# using the result fetched in cursor, create a new dataframe
new_df = pd.DataFrame(data,columns=columns)
new_df # shows all the values inserted - 7 columns

,neo_reference_id,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body
0,2415949,2024-01-02,57205.895120,0.337254,5.045241e+07,131.191622,Earth
1,3160747,2024-01-02,38589.054833,0.132343,1.979817e+07,51.481268,Earth
2,3309828,2024-01-02,56413.014352,0.167013,2.498473e+07,64.967910,Earth
3,3457842,2024-01-02,21891.118219,0.492051,7.360980e+07,191.407879,Earth
4,3553062,2024-01-02,31468.978359,0.235802,3.527551e+07,91.727075,Earth
...,...,...,...,...,...,...,...
9995,54527449,2025-04-13,50189.668328,0.153148,2.291054e+07,59.574384,Earth
9996,3152316,2025-04-14,55396.844838,0.394745,5.905302e+07,153.555841,Earth
9997,3441846,2025-04-14,20718.909111,0.243307,3.639815e+07,94.646263,Earth
9998,3763283,2025-04-14,51611.032647,0.395340,5.914196e+07,153.787090,Earth


# Step 5 : SQL Queries

In [20]:
import sqlite3
import pandas as pd

# Connect to your database
connection = sqlite3.connect('Asteroid_Data.db')
cursor = connection.cursor()

def show_query(query):
    df = pd.read_sql_query(query, connection)
    display(df)


In [21]:
query1 = '''
SELECT neo_reference_id, COUNT(*) AS approach_count
FROM close_approach
GROUP BY neo_reference_id
ORDER BY approach_count DESC
'''
show_query(query1)
# Count how many times each asteroid has approached Earth

,neo_reference_id,approach_count
0,54356994,5
1,54161317,5
2,3989284,5
3,3840791,5
4,3771633,5
...,...,...
7365,2006239,1
7366,2005660,1
7367,2005645,1
7368,2001685,1


In [22]:
query2 = '''
SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY avg_velocity DESC
'''
show_query(query2)


,neo_reference_id,avg_velocity
0,54392072,173071.830081
1,3803884,161520.897925
2,3790060,158279.864701
3,3425295,154336.407297
4,3092220,152218.220518
...,...,...
7365,54485079,3040.309519
7366,54459240,2149.995258
7367,3798020,1909.578168
7368,54440316,1610.566846


In [23]:
query3 = '''
SELECT neo_reference_id, MAX(relative_velocity_kmph) AS max_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY max_velocity DESC
LIMIT 10
'''
show_query(query3)


,neo_reference_id,max_velocity
0,54392072,173071.830081
1,3803884,161520.897925
2,3790060,158279.864701
3,3425295,154336.407297
4,3092220,152218.220518
5,3588900,151520.395439
6,54448599,150599.454097
7,54111253,150583.019419
8,3383901,149261.278634
9,2412977,148185.748026


In [24]:
query4 = '''
SELECT ca.neo_reference_id, COUNT(*) AS approach_count
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE a.is_potentially_hazardous_asteroid = 1
GROUP BY ca.neo_reference_id
HAVING COUNT(*) > 3
'''
show_query(query4)


,neo_reference_id,approach_count
0,2002340,4
1,2004034,4
2,2065679,4
3,2101955,4
4,2137126,4
...,...,...
90,54499464,4
91,54502451,4
92,54504542,4
93,54514905,9


In [25]:
query5 = '''
SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS count
FROM close_approach
GROUP BY month
ORDER BY count DESC
LIMIT 1
'''
show_query(query5)


,month,count
0,2024-10,902


In [26]:
query6 = '''
SELECT neo_reference_id, MAX(relative_velocity_kmph) AS fastest_speed
FROM close_approach
ORDER BY fastest_speed DESC
LIMIT 1
'''
show_query(query6)


,neo_reference_id,fastest_speed
0,54392072,173071.830081


In [27]:
query7 = '''
SELECT id, name, estimated_diameter_max_km
FROM asteroids
ORDER BY estimated_diameter_max_km DESC
'''
show_query(query7)


,id,name,estimated_diameter_max_km
0,2000887,887 Alinda (A918 AA),10.281109
1,2001685,1685 Toro (1948 OA),8.280184
2,2066146,66146 (1998 TU3),7.980681
3,2005660,5660 (1974 MA),4.989304
4,2533671,533671 (2014 LJ21),3.926811
...,...,...,...
9995,3556206,(2011 CQ1),0.002260
9996,54511953,(2024 YW8),0.002208
9997,54380194,(2023 RS),0.002042
9998,54473959,(2024 RC42),0.001986


In [28]:
query8 = '''
SELECT *
FROM close_approach
ORDER BY neo_reference_id, close_approach_date
'''
show_query(query8)


,neo_reference_id,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body
0,2000887,2025-01-08,29695.315959,0.082198,1.229661e+07,31.974916,Earth
1,2001685,2024-01-20,58226.721077,0.133063,1.990593e+07,51.761490,Earth
2,2002063,2024-03-31,30698.842824,0.120005,1.795253e+07,46.682031,Earth
3,2002063,2024-03-31,30698.842824,0.120005,1.795253e+07,46.682031,Earth
4,2002340,2024-11-02,93669.384688,0.397608,5.948137e+07,154.669680,Earth
...,...,...,...,...,...,...,...
9995,54533318,2024-12-08,9484.426629,0.121535,1.818143e+07,47.277241,Earth
9996,54533499,2025-04-03,38413.233597,0.194468,2.909198e+07,75.647998,Earth
9997,54533502,2024-05-12,14929.830125,0.045211,6.763441e+06,17.587005,Earth
9998,54533502,2024-05-12,14929.830125,0.045211,6.763441e+06,17.587005,Earth


In [29]:
query9 = '''
SELECT a.name, ca.close_approach_date, MIN(ca.miss_distance_km) AS closest_approach
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
GROUP BY a.id
ORDER BY closest_approach ASC
'''
show_query(query9)


,name,close_approach_date,closest_approach
0,(2024 XA),2024-12-01,7.726027e+03
1,(2024 LH1),2024-06-06,8.098256e+03
2,(2024 UG9),2024-10-30,8.849866e+03
3,(2025 BP6),2025-01-26,9.711954e+03
4,(2024 HA),2024-04-16,1.555513e+04
...,...,...,...
7365,(2019 LX),2024-09-23,7.476755e+07
7366,(2020 YH),2025-01-13,7.477857e+07
7367,(2023 GF),2025-04-10,7.478050e+07
7368,(2021 UQ3),2025-01-28,7.478989e+07


In [30]:
query10 = '''
SELECT DISTINCT a.name, ca.relative_velocity_kmph
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.relative_velocity_kmph > 50000
'''
show_query(query10)


,name,relative_velocity_kmph
0,415949 (2001 XY10),57205.895120
1,(2005 YQ96),56413.014352
2,(2017 YD8),68377.113479
3,(2019 KK5),74999.269000
4,613286 (2005 YQ96),56413.042562
...,...,...
3045,(2022 SW),76463.347450
3046,(2024 UZ9),61125.436104
3047,(2025 FW12),68150.761916
3048,(2003 GJ21),55396.844838


In [31]:
query11 = '''
SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS total
FROM close_approach
GROUP BY month
ORDER BY total DESC
'''
show_query(query11)


,month,total
0,2024-10,902
1,2024-09,853
2,2024-11,717
3,2024-04,709
4,2024-12,696
5,2024-03,653
6,2024-01,646
7,2025-03,636
8,2024-08,621
9,2024-02,607


In [32]:
query12 = '''
SELECT id, name, absolute_magnitude_h
FROM asteroids
ORDER BY absolute_magnitude_h ASC
LIMIT 1
'''
show_query(query12)


,id,name,absolute_magnitude_h
0,2000887,887 Alinda (A918 AA),13.81


In [33]:
query13 = '''
SELECT is_potentially_hazardous_asteroid, COUNT(*) AS count
FROM asteroids
GROUP BY is_potentially_hazardous_asteroid
'''
show_query(query13)


,is_potentially_hazardous_asteroid,count
0,0,9506
1,1,494


In [34]:
query14 = '''
SELECT a.name, ca.close_approach_date, ca.miss_distance_lunar
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.miss_distance_lunar < 1
ORDER BY ca.miss_distance_lunar
'''
show_query(query14)


,name,close_approach_date,miss_distance_lunar
0,(2024 XA),2024-12-01,0.020090
1,(2024 LH1),2024-06-06,0.021058
2,(2024 LH1),2024-06-06,0.021058
3,(2024 UG9),2024-10-30,0.023012
4,(2025 BP6),2025-01-26,0.025254
...,...,...,...
300,(2024 TA),2024-09-29,0.966317
301,(2024 UB1),2024-10-18,0.968912
302,(2024 JL3),2024-05-08,0.979100
303,(2024 RR8),2024-09-02,0.979981


In [35]:
query15 = '''
SELECT a.name, ca.close_approach_date, ca.astronomical
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.astronomical < 0.05
ORDER BY ca.astronomical
'''
show_query(query15)


,name,close_approach_date,astronomical
0,(2024 XA),2024-12-01,0.000052
1,(2024 LH1),2024-06-06,0.000054
2,(2024 LH1),2024-06-06,0.000054
3,(2024 UG9),2024-10-30,0.000059
4,(2025 BP6),2025-01-26,0.000065
...,...,...,...
3620,(2024 NZ1),2024-07-28,0.049882
3621,(2024 UF2),2024-11-04,0.049892
3622,(2024 QJ1),2024-07-27,0.049906
3623,(2006 GB1),2024-10-08,0.049947


# Bonus Query

In [36]:
# 💡 Bonus Query 1: Most frequently orbiting body (other than Earth)
Bonus_1 = ''' SELECT orbiting_body, COUNT(*) AS count
FROM close_approach
WHERE orbiting_body != 'Earth'
GROUP BY orbiting_body
ORDER BY count DESC '''
show_query(Bonus_1)
# 📌 Insight: Understand which planets or celestial bodies asteroids approach most after Earth.

,orbiting_body,count


In [37]:
# 💡 Bonus Query 2: Average miss distance by hazard type
Bonus_2 = '''
SELECT a.is_potentially_hazardous_asteroid, AVG(ca.miss_distance_km) AS avg_miss_distance
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
GROUP BY a.is_potentially_hazardous_asteroid
'''
show_query(Bonus_2)
# 📌 Insight: Do hazardous asteroids pass closer to Earth on average?


,is_potentially_hazardous_asteroid,avg_miss_distance
0,0,3.004464e+07
1,1,3.640202e+07


In [38]:
# 💡 Bonus Query 3: Top 5 closest approaches to Earth
Bonus_3 = '''
SELECT a.name, ca.close_approach_date, ca.miss_distance_km
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
ORDER BY ca.miss_distance_km ASC
LIMIT 5
'''
show_query(Bonus_3)
# 📌 Insight: Identify the closest asteroid encounters with Earth.

,name,close_approach_date,miss_distance_km
0,(2024 XA),2024-12-01,7726.026876
1,(2024 LH1),2024-06-06,8098.256296
2,(2024 LH1),2024-06-06,8098.256296
3,(2024 UG9),2024-10-30,8849.865914
4,(2025 BP6),2025-01-26,9711.953560


In [39]:
# 💡 Bonus Query 4: Total unique hazardous asteroids
Bonus_4 = '''
SELECT COUNT(DISTINCT id) AS hazardous_asteroid_count
FROM asteroids
WHERE is_potentially_hazardous_asteroid = 1
'''
show_query(Bonus_4)
# 📌 Insight: How many unique asteroids are potentially hazardous?

,hazardous_asteroid_count
0,383


In [40]:
# 💡 Bonus Query 5: Asteroids that passed within 1 LD more than once
Bonus_5 = '''
SELECT ca.neo_reference_id, a.name, COUNT(*) AS close_pass_count
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.miss_distance_lunar < 1
GROUP BY ca.neo_reference_id
HAVING COUNT(*) > 1
ORDER BY close_pass_count DESC
'''
show_query(Bonus_5)
# 📌 Insight: Detect recurring close encounters by the same asteroid.

,neo_reference_id,name,close_pass_count
0,54494154,(2024 TK22),6
1,54525636,(2025 FE18),4
2,54525458,(2025 FZ15),4
3,54523430,(2025 EV3),4
4,54520041,(2025 DW),4
5,54506551,(2024 XP16),4
6,54502879,(2024 XH1),4
7,54499834,(2024 WO),4
8,54499742,(2024 WA),4
9,54499725,(2024 VW4),4


# Streamlit Application

In [41]:
!pip install streamlit streamlit_option_menu  # installing streamlit and streamlit_option_menu packages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 12.4 MB/s eta 0:00:00


In [1]:
%%writefile nasa_project.py

import streamlit as st
import sqlite3
import pandas as pd
from datetime import datetime

# Streamlit UI setup
st.set_page_config(layout='wide')

# Title and intro
st.title("🌌 NASA NEO Tracking & Insights Dashboard")
st.markdown("""
Explore asteroid data, approach speeds, distances, and hazard insights using SQL-powered queries.
""")

# Connect to the database
conn = sqlite3.connect("Asteroid_Data.db")
cursor = conn.cursor()

# Helper function to run and display SQL queries
def show_query(query):
    df = pd.read_sql_query(query, conn)
    st.dataframe(df)

# Sidebar filters
st.sidebar.header("📊 Query & Filters")
query_option = st.sidebar.selectbox("Select a Query", [
    "1. Count asteroid approaches",
    "2. Average velocity per asteroid",
    "3. Top 10 fastest asteroids",
    "4. Hazardous asteroids > 3 approaches",
    "5. Month with most approaches",
    "6. Fastest ever approach",
    "7. Sort by max estimated diameter",
    "8. Closest approach getting nearer over time",
    "9. Closest approach date & distance",
    "10. Velocity > 50,000 km/h",
    "11. Approaches per month",
    "12. Brightest asteroid (lowest magnitude)",
    "13. Hazardous vs Non-hazardous count",
    "14. Asteroids < 1 LD",
    "15. Asteroids < 0.05 AU",
    "Bonus 1: Orbiting bodies (non-Earth)",
    "Bonus 2: Avg miss distance by hazard type",
    "Bonus 3: Top 5 closest approaches",
    "Bonus 4: Count of hazardous asteroids",
    "Bonus 5: Frequent <1 LD asteroids"
])

# All Queries defined here -- Chatgpt suggestions i don't know what it means
queries = {
    "1. Count asteroid approaches": '''
        SELECT neo_reference_id, COUNT(*) AS approach_count
        FROM close_approach
        GROUP BY neo_reference_id
        ORDER BY approach_count DESC
    ''',
    "2. Average velocity per asteroid": '''
        SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
        FROM close_approach
        GROUP BY neo_reference_id
        ORDER BY avg_velocity DESC
    ''',
    "3. Top 10 fastest asteroids": '''
        SELECT neo_reference_id, MAX(relative_velocity_kmph) AS max_velocity
        FROM close_approach
        GROUP BY neo_reference_id
        ORDER BY max_velocity DESC
        LIMIT 10
    ''',
    "4. Hazardous asteroids > 3 approaches": '''
        SELECT ca.neo_reference_id, COUNT(*) AS approach_count
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        WHERE a.is_potentially_hazardous_asteroid = 1
        GROUP BY ca.neo_reference_id
        HAVING COUNT(*) > 3
    ''',
    "5. Month with most approaches": '''
        SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS count
        FROM close_approach
        GROUP BY month
        ORDER BY count DESC
        LIMIT 1
    ''',
    "6. Fastest ever approach": '''
        SELECT neo_reference_id, MAX(relative_velocity_kmph) AS fastest_speed
        FROM close_approach
        ORDER BY fastest_speed DESC
        LIMIT 1
    ''',
    "7. Sort by max estimated diameter": '''
        SELECT id, name, estimated_diameter_max_km
        FROM asteroids
        ORDER BY estimated_diameter_max_km DESC
    ''',
    "8. Closest approach getting nearer over time": '''
        SELECT *
        FROM close_approach
        ORDER BY neo_reference_id, close_approach_date
    ''',
    "9. Closest approach date & distance": '''
        SELECT a.name, ca.close_approach_date, MIN(ca.miss_distance_km) AS closest_approach
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        GROUP BY a.id
        ORDER BY closest_approach ASC
    ''',
    "10. Velocity > 50,000 km/h": '''
        SELECT DISTINCT a.name, ca.relative_velocity_kmph
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        WHERE ca.relative_velocity_kmph > 50000
    ''',
    "11. Approaches per month": '''
        SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS total
        FROM close_approach
        GROUP BY month
        ORDER BY total DESC
    ''',
    "12. Brightest asteroid (lowest magnitude)": '''
        SELECT id, name, absolute_magnitude_h
        FROM asteroids
        ORDER BY absolute_magnitude_h ASC
        LIMIT 1
    ''',
    "13. Hazardous vs Non-hazardous count": '''
        SELECT is_potentially_hazardous_asteroid, COUNT(*) AS count
        FROM asteroids
        GROUP BY is_potentially_hazardous_asteroid
    ''',
    "14. Asteroids < 1 LD": '''
        SELECT a.name, ca.close_approach_date, ca.miss_distance_lunar
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        WHERE ca.miss_distance_lunar < 1
        ORDER BY ca.miss_distance_lunar
    ''',
    "15. Asteroids < 0.05 AU": '''
        SELECT a.name, ca.close_approach_date, ca.miss_distance_au
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        WHERE ca.miss_distance_au < 0.05
        ORDER BY ca.miss_distance_au
    ''',
    "Bonus 1: Orbiting bodies (non-Earth)": '''
        SELECT orbiting_body, COUNT(*) AS count
        FROM close_approach
        WHERE orbiting_body != 'Earth'
        GROUP BY orbiting_body
        ORDER BY count DESC
    ''',
    "Bonus 2: Avg miss distance by hazard type": '''
        SELECT a.is_potentially_hazardous_asteroid, AVG(ca.miss_distance_km) AS avg_miss_distance
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        GROUP BY a.is_potentially_hazardous_asteroid
    ''',
    "Bonus 3: Top 5 closest approaches": '''
        SELECT a.name, ca.close_approach_date, ca.miss_distance_km
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        ORDER BY ca.miss_distance_km ASC
        LIMIT 5
    ''',
    "Bonus 4: Count of hazardous asteroids": '''
        SELECT COUNT(DISTINCT id) AS hazardous_asteroid_count
        FROM asteroids
        WHERE is_potentially_hazardous_asteroid = 1
    ''',
    "Bonus 5: Frequent <1 LD asteroids": '''
        SELECT ca.neo_reference_id, a.name, COUNT(*) AS close_pass_count
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        WHERE ca.miss_distance_lunar < 1
        GROUP BY ca.neo_reference_id
        HAVING COUNT(*) > 1
        ORDER BY close_pass_count DESC
    '''
}

# Run selected query
st.subheader(query_option)
show_query(queries[query_option])

# 🔍 Advanced Filters Section
st.header("📌 Filter Asteroid Approaches")

# User input widgets
selected_date = st.date_input("Select Close Approach Date (after)", datetime(2000, 1, 1))
min_au = st.slider("Minimum Astronomical Units (AU)", 0.0, 1.0, 0.0, 0.01)
max_au = st.slider("Maximum Astronomical Units (AU)", 0.0, 1.0, 0.05, 0.01)
min_ld = st.slider("Minimum Lunar Distance (LD)", 0.0, 100.0, 0.0, 1.0)
max_ld = st.slider("Maximum Lunar Distance (LD)", 0.0, 100.0, 10.0, 1.0)
min_velocity = st.slider("Minimum Relative Velocity (km/h)", 0.0, 100000.0, 0.0, 1000.0)
max_velocity = st.slider("Maximum Relative Velocity (km/h)", 0.0, 100000.0, 50000.0, 1000.0)
min_diameter = st.slider("Minimum Estimated Diameter (km)", 0.0, 50.0, 0.0, 0.1)
max_diameter = st.slider("Maximum Estimated Diameter (km)", 0.0, 50.0, 5.0, 0.1)
hazardous = st.selectbox("Hazardous?", ["Both", "Yes", "No"])

# Filter query
filter_query = f'''
SELECT a.name, ca.close_approach_date, ca.relative_velocity_kmph, ca.miss_distance_km, ca.miss_distance_lunar,
       a.estimated_diameter_min_km, a.estimated_diameter_max_km, a.is_potentially_hazardous_asteroid
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE date(ca.close_approach_date) >= date('{selected_date}')
  AND ca.miss_distance_km BETWEEN {min_au} AND {max_au}
  AND ca.miss_distance_lunar BETWEEN {min_ld} AND {max_ld}
  AND ca.relative_velocity_kmph BETWEEN {min_velocity} AND {max_velocity}
  AND a.estimated_diameter_max_km BETWEEN {min_diameter} AND {max_diameter}
'''
if hazardous == "Yes":
    filter_query += " AND a.is_potentially_hazardous_asteroid = 1"
elif hazardous == "No":
    filter_query += " AND a.is_potentially_hazardous_asteroid = 0"

st.subheader("Filtered Results")
show_query(filter_query)

# Launch instructions for Colab
st.markdown("""
---
### 🔗 Launch from Google Colab
python
!wget -q -O - ipv4.icanhazip.com   # this command will generate a password for you (copy that)
!streamlit run nasa_project.py & npx localtunnel --port 8501

Then when prompted:
- Enter y to proceed
- A link like https://fruity-aliens-unite.loca.lt/ will be generated
- Paste it in your browser
- It will ask for a password (paste the one copied earlier)
- You’ll be redirected to your Streamlit app ✅
---
""")

Writing nasa_project.py


In [2]:
!wget -q -O - ipv4.icanhazip.com   # this command will generate a password for you(copy that)

34.86.213.57


In [ ]:
!streamlit run nasa_project.py & npx localtunnel --port 8501


/bin/bash: line 1: streamlit: command not found
⠙⠹⠸⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹your url is: https://bumpy-moons-raise.loca.lt
